### 自定义数据读取
在Dataloader_default中我们学会采用torchvision.datasets.ImageFolder这个接口来读取图像数据，**该接口默认你的训练数据是按照一个类别存放在一个文件夹下。**但是有些情况下你的图像数据不是这样维护的，比如一个文件夹下面各个类别的图像数据都有，同时用一个对应的标签文件，比如txt文件来维护图像和标签的对应关系，在这种情况下就不能用torchvision.datasets.ImageFolder来读取数据了，需要自定义一个数据读取接口。

### torchvision.datasets.ImageFolder类源码

In [ ]:
class ImageFolder(data.Dataset):
    """A generic data loader where the images are arranged in this way: ::

        root/dog/xxx.png
        root/dog/xxy.png
        root/dog/xxz.png

        root/cat/123.png
        root/cat/nsdf3.png
        root/cat/asd932_.png

    参数:
        - root：在root指定的路径下寻找图片
        - transform：对PIL Image进行的转换操作，transform的输入是使用loader读取图片的返回对象
        - target_transform：对label的转换
        - loader：给定路径后如何读取图片，默认读取为RGB格式的PIL Image对象

     属性:
        classes (list):类名列表
        class_to_idx (dict):类名和类标签字典 {class_name:class_index,...}.
        imgs (list): 图像路径和标签元组的列表 [(image path,class_index)...]
    """


    def __init__(self, root, transform=None, target_transform=None,
                 loader=default_loader):
        classes, class_to_idx = find_classes(root)
        imgs = make_dataset(root, class_to_idx)
        if len(imgs) == 0:
            raise(RuntimeError("Found 0 images in subfolders of: " + root + "\n"
                               "Supported image extensions are: " + ",".join(IMG_EXTENSIONS)))

        self.root = root
        self.imgs = imgs
        self.classes = classes
        self.class_to_idx = class_to_idx
        self.transform = transform
        self.target_transform = target_transform
        self.loader = loader

    def __getitem__(self, index):
        """
        Args:
            index (int): Index

        Returns:
            tuple: (image, target) where target is class_index of the target class.
        """
        path, target = self.imgs[index]
        img = self.loader(path)
        if self.transform is not None:
            img = self.transform(img)
        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

    def __len__(self):
        return len(self.imgs)

看起来很复杂，其实非常简单。继承的类是torch.utils.data.Dataset，主要包含三个方法：
- 初始化\_\_init\_\_，
- 获取图像\_\_getitem\_\_，
- 数据集数量 \_\_len\_\_。

\_\_init\_\_方法中先通过find_classes函数得到分类的类别名（classes）和类别名与数字类别的映射关系字典（class_to_idx）。然后通过make_dataset函数得到imags，这个imags是一个列表，其中每个值是一个tuple，每个tuple包含两个元素：图像路径和标签。剩下的就是一些赋值操作了。

在\_\_getitem\_\_方法中最重要的就是 img = self.loader(path)这行，表示数据读取，可以从\_\_init\_\_方法中看出self.loader采用的是default_loader，这个default_loader的核心就是用python的PIL库的Image模块来读取图像数据。


**稍微看下default_loader函数，该函数主要分两种情况调用两个函数，一般采用pil_loader函数。**

In [ ]:
def pil_loader(path):
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')

def accimage_loader(path):
    import accimage
    try:
        return accimage.Image(path)
    except IOError:
        # Potentially a decoding problem, fall back to PIL.Image
        return pil_loader(path)

def default_loader(path):
    from torchvision import get_image_backend
    if get_image_backend() == 'accimage':
        return accimage_loader(path)
    else:

看懂了ImageFolder这个类，就可以自定义一个你自己的数据读取接口了。

**首先在PyTorch中和数据读取相关的类基本都要继承一个基类：torch.utils.data.Dataset。然后再改写其中的__init__、__len__、__getitem__等方法即可。**

### 图片分类数据集

图片分类数据集应该分为train,val,两个目录，每个类别的所有图片放到该类名对应的目录下，可以直接调用torchvision.datasets.ImageFolder类分装数据集

### 图片分割数据集

In [ ]:
import os

### 训练集，测试集lst文档准备

In [ ]:
root_dir="./data_segmentation"
sub_dirs=os.listdir(root_dir)
print(sub_dirs)

train=[]

#['raw', 'gt']
for idx,sub_dir in enumerate(sub_dirs):
    phasenames=os.listdir(os.path.join(root_dir,sub_dir))
    
    #['train']
    for phasename in phasenames:
        pwd_dir=os.path.join(root_dir,sub_dir,phasename)
        
        if phasename=='train':
            
            if idx==0:
                
                train_raw_filenames=[sub_dir+'/'+phasename+'/'+item for item in os.listdir(pwd_dir)]
            if idx==1:
                
                train_gt_filenames=[sub_dir+'/'+phasename+'/'+item for item in os.listdir(pwd_dir)]
        if phasename=='test':
            test_filenames=[sub_dir+'/'+phasename+'/'+item for item in os.listdir(pwd_dir)]
    
# print(train_raw_filenames)
# print(train_gt_filenames)
# print(test_filenames)

for train_raw,train_gt in zip(train_raw_filenames,train_gt_filenames):
    train.append(train_raw+'\t'+train_gt)
    
print(train[0])

In [ ]:
with open(os.path.join(root_dir,'list/train.lst'),'a+') as train_file:
    for item in train:
        train_file.write(item+os.linesep)

In [ ]:
with open(os.path.join(root_dir,'list/test.lst'),'a+') as test_file:
    for item in test_filenames:
        test_file.write(item+os.linesep)

### 训练集，测试集lst文档内容
#### list/train.lst：(原图路径名 制表符 gt图路径名)
raw/train/0d1a9caf4350_16.jpg	gt/train/0d1a9caf4350_16_mask.gif

raw/train/0cdf5b5d0ce1_01.jpg	gt/train/0cdf5b5d0ce1_01_mask.gif

raw/train/0cdf5b5d0ce1_02.jpg	gt/train/0cdf5b5d0ce1_02_mask.gif

raw/train/0cdf5b5d0ce1_03.jpg	gt/train/0cdf5b5d0ce1_03_mask.gif

raw/train/0cdf5b5d0ce1_04.jpg	gt/train/0cdf5b5d0ce1_04_mask.gif

raw/train/0cdf5b5d0ce1_05.jpg	gt/train/0cdf5b5d0ce1_05_mask.gif

raw/train/0cdf5b5d0ce1_06.jpg	gt/train/0cdf5b5d0ce1_06_mask.gif

raw/train/0cdf5b5d0ce1_07.jpg	gt/train/0cdf5b5d0ce1_07_mask.gif

#### list/test.lst：(原图路径名)

### 自定义Dataset

In [12]:
from torch.utils import data
from PIL import Image
import os
import cv2
import numpy as np

In [ ]:
#img_raw=cv2.imread('./data_segmentation/raw/train/0cdf5b5d0ce1_07.jpg',cv2.IMREAD_ANYCOLOR)
#img_label=cv2.imread('./data_segmentation/gt/train/0cdf5b5d0ce1_07_mask.jpg',cv2.IMREAD_ANYCOLOR)
gif = cv2.VideoCapture('./data_segmentation/gt/train/0cdf5b5d0ce1_07_mask.gif')
ret,frame = gif.read() # ret=True if it finds a frame else False. Since your gif contains only one frame, the next read() will give you ret=False
img = Image.fromarray(frame)
img_label = img.convert('L')
img_label=np.asarray(img_label)
#print(img_raw.shape)
print(type(img_label))
print(img_label.shape)
#print(type(img_raw))
#cv2.imshow('raw',img_raw)
cv2.imshow('label',img_label)
cv2.waitKey(0)
cv2.destroyAllWindows()

<class 'numpy.ndarray'>
(1280, 1918)


In [ ]:
class MyDataset(data.Dataset):
    def __init__(self,root,list_path,ignore_label=255):
        super(MyDataset,self).__init__()
        
        self.root=root#'data_segmentation/'
        self.list_path=list_path
        
        #list of [raw image path,label image path]
        self.img_list=[line.strip().split() for line in open(root+list_path)]
        
        #字典的列表，每个字典记录1个样本的信息：img, label, name...
        self.files=self.read_files()
    
    
    def __len__(self):
        return len(self.files)
        
    def __getitem__(self,index):
        '''
        这个方法是必须要有的，用于按照索引读取每个元素的具体内容
        :param index:
        :return:
        #测试集只有原图像
        #返回:原图像变换后的image的np数组，图像size的np数组，样本文件名
        #训练集
        #返回:原图像变换后的image的np数组，label图像变换后的label image的np数组，原图像size的np数组，样本文件名
        return很关键，return回哪些内容，那么我们在训练时循环读取每个batch时，就能获得哪些内容
        '''
        item=self.files[index]
        
        img_path=item['img']
        
        image=cv2.imread(os.join(self.root,img_path))
        size=image.shape
                         
        if 'test' in self.list_path:
            #对raw image做变换
            image=self.input_transform(image)
            return image.copy(),np.array(size),item['name']
        else
            image=self.input_transform(image)
            label_path=item['label']
            #
            #label=cv2.imread(os.join(self.root,label_path))
            gif=cv2.VideoCapture(os.join(self.root,img_path))
            ret,frame = gif.read() # ret=True if it finds a frame else False. Since your gif contains only one frame, the next read() will give you ret=False
            label = Image.fromarray(frame).convert('L')
            label=np.asarray(img_label)
            return image.copy(),label.copy(),np.array(size),item['name']
    
    def input_transform(image):
        '''
        转换为np.float32类型,归一化，BGR->RGB，通道放在第1维
        '''
        image=image.astype(np.float32)[:,:,::-1]
        image=image/255.0
        image=image.transpose((2,0,1))
        return image
        
    def read_files(self):
        files=[]
        if 'test' in list_path:
            for item in self.img_list:
                img_path=item
                name=os.path.splitext(os.path.basename(img_path[0]))[0]
                files.append({
                    'img':img_path,
                    'name':name
                })
        else:
            for item in self.img_list:
                img_path,label_path=item
                name=os.path.splitext(os.path.basename(label_path))[0]
                files.append({
                    'img':img_path,
                    'label':label_path,
                    'name':name,
                    'weight':1
                })